# Final Project for CS 6.338
# Emily Crabb

# Parallel molecular dyanmics simulation implementation in Julia

some sort of description of project !!!!!!!!

Sources:

https://people.sc.fsu.edu/~jburkardt/py_src/md/md.py

https://www.saylor.org/site/wp-content/uploads/2011/06/MA221-6.1.pdf


To do (in no particular order):

Add parallelism - high priority

Convert to script - important for analysis

Allow user to specify values of parameters - lower priority

Read initial conditions from file - lower priority

Save data to file - some already done but isn't enough to restart probably - lower priority but is easy

Display output - manipulate doesn't work - medium priority


In [1]:
# Generate strengths of interactions between particle types
# Very simple but user could replace with anything they wanted

function gen_interaction(num_part_types)
    interaction_params = zeros(Float64, num_part_types,num_part_types)
    for i=1:num_part_types
        for j = 1:num_part_types
            if (i==j) # Self-interaction is randomly repulsive
                interaction_params[i,j] = -1*rand(Float64)
            elseif (i<j) # Others randomly attractive
                val = rand(Float64)
                interaction_params[i,j] = val
                interaction_params[j,i] = val
            end
        end        
    end
    
    return interaction_params
end

gen_interaction (generic function with 1 method)

In [27]:
# Global constants 
# If convert to script, some should be user inputs

const number_of_steps = 10000; # Number of steps to execute in simulation
const dim = 2; # Dimensions of simulation
const box_size = 10; # Size of one side of box
const finite_box = true; # Whether box if finite or just where particles are initially placed
const periodic = true; # Whether simulation is periodic
const part_num = 100; # Number of particles in simulation
const dt = 0.01; # Time step
const num_part_types = 2; # Number of types of particles
const interaction_params = gen_interaction(num_part_types); # Interations parameters for types of particles
const mass_parts = rand(Float64, num_part_types); # Masses of types of particles
const save_interval = 10; # How often save position
const save_data = false; # Whether to save data to file so can restart

In [3]:
# Initialize position, velocity, acceleration, and particle type
# Currently start with zero velocity and acceleration
# Currently start with random positions and randomly assigned particle types

function initialize(part_num, dim, box_size, num_part_types)
    
    pos = box_size*rand(Float64, part_num, dim) # Initialized to be randomly placed within a box
    
    vel = zeros(Float64, part_num, dim) # Initialized to zero
    acc = zeros(Float64, part_num, dim) # Initialized to zero
    
    part_types = rand(1:num_part_types, part_num) # Randomly assign type of each particle
        
    return pos, vel, acc, part_types
end

initialize (generic function with 1 method)

In [4]:
# Update position, velocity, and acceleration using Velocity Verlet Algorithm
# Currently system size is infinite (positions not constrained)

function step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)
    
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for j = 1:dim # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end
    
    return pos, vel, acc
end

step_update (generic function with 1 method)

In [34]:
# Find force on each particle
# 1/r^2 interactions currently
# Again, very simple but user can replace with anything they want

function find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts)
    
    force = zeros(part_num, dim)
    
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j = 1:dim # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    if (pos[i,j] > pos[k,j])
                       int_strength = -1*int_strength # Reverses direction of force if positions flopped
                    end

                    # Find distance between particles
                    dist = 0.0
                    if (periodic) # If periodic, check whether a periodic distance is shorter
                        dist = abs(pos[i,j] - pos[k,j])
                        if (pos[i,j] < pos[k,j])
                            new_dist = abs(box_size + pos[i,j] - pos[k,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        else
                            new_dist = abs(box_size + pos[k,j] - pos[i,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        end
                        else # Otherwise, just regular distance
                        dist = abs(pos[i,j] - pos[k,j])
                    end
                    
                    force[i,j] = int_strength / dist^2 # 1/r^2 interaction
                end
            end
        end
    end
    
    return force
end

find_force (generic function with 1 method)

In [6]:
# Write each variable to its own output file in current diretory
# May need to add more imfo if want to restart

function write_output(part_num, dim, part_types, interaction_params, mass_parts, dt, saved_positions, saved_velocities, saved_accelerations, saved_forces) 
    writedlm("part_num.txt", part_num)
    writedlm("dim.txt", dim)
    writedlm("part_types.txt", part_types)
    writedlm("interaction_params.txt", interaction_params)
    writedlm("mass_parts.txt", mass_parts)
    writedlm("dt.txt", dt)
    writedlm("saved_positions.txt", saved_positions)
    writedlm("saved_velocities.txt", saved_velocities)
    writedlm("saved_accelerations.txt", saved_accelerations)
    writedlm("saved_forces.txt", saved_forces)
end

write_output (generic function with 1 method)

In [35]:
# Main - where program executes
# Convert to main(args) if convert to script

# Set up matrices to save positions, velocities, and accelerations into
num_pos = floor(Int, number_of_steps/save_interval)+1 # Number of positions to save
saved_positions = zeros(num_pos, part_num, dim) # Save positions with specified frequency
if (save_data) # Only save vel, acc, and force if plan on saving to file
    saved_velocities = zeros(num_pos, part_num, dim) # Save velocities with specified frequency
    saved_accelerations = zeros(num_pos, part_num, dim) # Save accelerations with specified frequency
    saved_forces = zeros(num_pos, part_num, dim) # Save forces with specified frequency
end
save_index = 1 # Keep track of index so can save positions with specified frequency

pos, vel, acc, part_types = initialize(part_num, dim, box_size, num_part_types) # Initialize
force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts) # Find forces on particles
saved_positions[save_index,:,:] = pos # Save position
if (save_data) # Only save vel, acc, and force if plan on saving to file
    saved_velocities[save_index,:,:] = vel # Save velocity
    saved_accelerations[save_index,:,:] = acc # Save accelerations
    saved_forces[save_index,:,:] = force # Save forces
end
save_index += 1 # Increment index

for i = 1:number_of_steps
    pos, vel, acc = step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic) # Update
    force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts) # Find new forces
    
    if (i % save_interval == 0) # If are saving this time step
        saved_positions[save_index,:,:] = pos # Save position
        if (save_data) # Only save vel, acc, and force if plan on saving to file
            saved_velocities[save_index,:,:] = vel # Save velocity
            saved_accelerations[save_index,:,:] = acc # Save accelerations
            saved_forces[save_index,:,:] = force # Save forces
        end
        save_index += 1 # Increment index
    end
end

if (save_data) # If save data
    write_output(part_num, dim, part_types, interaction_params, mass_parts, dt, saved_positions, saved_velocities, saved_accelerations, saved_forces)
end

In [36]:
#Pkg.add("Plots")
using Plots
plotly()

Plots.PlotlyBackend()

In [37]:
#plot(pos[:,1],pos[:,2], seriestype=:scatter)

In [38]:
#Pkg.add("Interact")
using Interact

In [39]:
#@manpulate for index=1:num_pos
index = 1

    plot(saved_positions[index,:,1],saved_positions[index,:,2], seriestype=:scatter)    
#end

In [40]:
index = 50    
plot(saved_positions[index,:,1],saved_positions[index,:,2], seriestype=:scatter)    
